<a href="https://colab.research.google.com/github/MarsZDF/argentum-v2/blob/master/argentum_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Argentum StateDiff Demo

Simple state tracking for AI agents - see exactly how your agent's state evolves over time.

This notebook demonstrates how to use Argentum to track and visualize state changes in AI agents across different frameworks.

## Installation

In [ ]:
# Install from source (for demo)
!pip install git+https://github.com/MarsZDF/argentum-v2.git

# Or install from PyPI when published:
# !pip install argentum

## Basic Usage

Let's start with a simple example of tracking state changes:

In [ ]:
from argentum import StateDiffTracker

# Create tracker
tracker = StateDiffTracker()

# Track simple state changes
tracker.capture("initial", {
    "thoughts": "I need to help the user",
    "plan": ["understand", "research"],
    "status": "thinking"
})

tracker.capture("updated", {
    "thoughts": "I found the answer", 
    "plan": ["understand", "research", "respond"],
    "status": "ready",
    "answer": "The solution is X"
})

# See what changed
tracker.show_diff("initial", "updated")

## LangChain-Style Agent Simulation

This example simulates how LangChain agents typically manage state with intermediate steps:

In [ ]:
# Simulate a LangChain agent workflow
tracker_lc = StateDiffTracker()

# Agent starts
tracker_lc.capture("start", {
    "input": "What's the capital of France?",
    "intermediate_steps": [],
    "agent_scratchpad": "",
    "chat_history": []
})

# Agent uses search tool
tracker_lc.capture("after_search", {
    "input": "What's the capital of France?",
    "intermediate_steps": [
        {"action": "search", "action_input": "capital France", "observation": "Paris is the capital of France"}
    ],
    "agent_scratchpad": "I searched for the capital of France and found it's Paris.",
    "chat_history": []
})

# Agent completes
tracker_lc.capture("complete", {
    "input": "What's the capital of France?",
    "intermediate_steps": [
        {"action": "search", "action_input": "capital France", "observation": "Paris is the capital of France"}
    ],
    "agent_scratchpad": "I searched for the capital of France and found it's Paris.",
    "chat_history": [],
    "output": "The capital of France is Paris."
})

print("=== LangChain Agent Evolution ===")
tracker_lc.show_diff("start", "after_search")
print()
tracker_lc.show_diff("after_search", "complete")

## CrewAI-Style Flow Simulation

This demonstrates how CrewAI flows manage state across multiple agents:

In [ ]:
# Simulate CrewAI flow state
tracker_crew = StateDiffTracker()

# Research flow starts
tracker_crew.capture("flow_start", {
    "topic": "AI safety",
    "depth": "medium",
    "sections": ["introduction", "risks", "solutions"],
    "completed_sections": [],
    "current_agent": "researcher",
    "results": {}
})

# After research phase
tracker_crew.capture("research_done", {
    "topic": "AI safety", 
    "depth": "medium",
    "sections": ["introduction", "risks", "solutions"],
    "completed_sections": ["introduction", "risks"],
    "current_agent": "writer", 
    "results": {
        "introduction": "AI safety is critical for ensuring beneficial outcomes as AI systems become more powerful and autonomous.",
        "risks": "Key risks include alignment problems, capability overhang, and unintended optimization targets."
    }
})

# Final report
tracker_crew.capture("flow_complete", {
    "topic": "AI safety",
    "depth": "medium", 
    "sections": ["introduction", "risks", "solutions"],
    "completed_sections": ["introduction", "risks", "solutions"],
    "current_agent": "reviewer",
    "results": {
        "introduction": "AI safety is critical for ensuring beneficial outcomes as AI systems become more powerful and autonomous.",
        "risks": "Key risks include alignment problems, capability overhang, and unintended optimization targets.",
        "solutions": "Proposed solutions include interpretability research, robustness testing, and gradual capability deployment."
    },
    "final_report": "# AI Safety Report\n\nThis comprehensive analysis covers the current state of AI safety research and recommendations for future development."
})

print("=== CrewAI Flow Evolution ===")
tracker_crew.show_diff("flow_start", "research_done")
print()
tracker_crew.show_diff("research_done", "flow_complete")

## Complex Agent State Example

Let's simulate a more complex agent with nested state and multiple tool calls:

In [ ]:
# Complex agent state simulation
tracker_complex = StateDiffTracker()

# Initial planning state
tracker_complex.capture("planning", {
    "goal": "Analyze quarterly sales data and create report",
    "context": {
        "user_id": "user123",
        "session_id": "sess456",
        "permissions": ["read_sales", "write_reports"]
    },
    "workflow": {
        "current_step": "planning",
        "completed_steps": [],
        "next_steps": ["data_collection", "analysis", "report_generation"]
    },
    "memory": {
        "working_data": {},
        "insights": [],
        "errors": []
    }
})

# After data collection
tracker_complex.capture("data_collected", {
    "goal": "Analyze quarterly sales data and create report",
    "context": {
        "user_id": "user123",
        "session_id": "sess456",
        "permissions": ["read_sales", "write_reports"]
    },
    "workflow": {
        "current_step": "data_collection",
        "completed_steps": ["planning"],
        "next_steps": ["analysis", "report_generation"]
    },
    "memory": {
        "working_data": {
            "q1_sales": 125000,
            "q2_sales": 143000,
            "q3_sales": 167000,
            "records_count": 1247
        },
        "insights": ["Strong growth trend observed"],
        "errors": []
    },
    "tool_calls": [
        {"tool": "database_query", "query": "SELECT * FROM sales WHERE quarter IN (1,2,3)", "result": "success"}
    ]
})

# After analysis
tracker_complex.capture("analysis_complete", {
    "goal": "Analyze quarterly sales data and create report",
    "context": {
        "user_id": "user123",
        "session_id": "sess456",
        "permissions": ["read_sales", "write_reports"]
    },
    "workflow": {
        "current_step": "analysis",
        "completed_steps": ["planning", "data_collection"],
        "next_steps": ["report_generation"]
    },
    "memory": {
        "working_data": {
            "q1_sales": 125000,
            "q2_sales": 143000,
            "q3_sales": 167000,
            "records_count": 1247,
            "growth_rate": 0.168,
            "trend": "increasing"
        },
        "insights": [
            "Strong growth trend observed",
            "Q3 shows 16.8% growth vs Q2",
            "Consistent quarter-over-quarter improvement"
        ],
        "errors": []
    },
    "tool_calls": [
        {"tool": "database_query", "query": "SELECT * FROM sales WHERE quarter IN (1,2,3)", "result": "success"},
        {"tool": "statistics_calc", "operation": "growth_rate", "result": 0.168}
    ]
})

print("=== Complex Agent State Evolution ===")
print("\n1. Planning → Data Collection:")
tracker_complex.show_diff("planning", "data_collected")
print("\n2. Data Collection → Analysis Complete:")
tracker_complex.show_diff("data_collected", "analysis_complete")

## Export and Analysis

Argentum allows you to export state snapshots for further analysis:

In [ ]:
# List all snapshots across different trackers
print("Available snapshots:")
print(f"Basic tracker: {tracker.list_snapshots()}")
print(f"LangChain tracker: {tracker_lc.list_snapshots()}")
print(f"CrewAI tracker: {tracker_crew.list_snapshots()}")
print(f"Complex tracker: {tracker_complex.list_snapshots()}")

# Export different workflows for analysis
tracker_crew.export_json("crewai_flow_debug.json")
tracker_complex.export_json("complex_agent_debug.json")

print("\n✅ Exported state traces:")
print("- crewai_flow_debug.json (CrewAI workflow)")
print("- complex_agent_debug.json (Complex agent state)")
print("\nYou can download these files from the Colab files panel for analysis.")

## Try It Yourself!

Now create your own agent state tracking. Try different scenarios:

In [ ]:
# Your playground - try tracking your own agent states!
my_tracker = StateDiffTracker()

# Example 1: Simple chatbot state
my_tracker.capture("user_message", {
    "conversation_id": "conv123",
    "messages": [{"role": "user", "content": "Hello!"}],
    "intent": "greeting",
    "mood": "neutral"
})

my_tracker.capture("bot_response", {
    "conversation_id": "conv123",
    "messages": [
        {"role": "user", "content": "Hello!"},
        {"role": "assistant", "content": "Hi there! How can I help you today?"}
    ],
    "intent": "greeting",
    "mood": "friendly",
    "response_generated": True
})

my_tracker.show_diff("user_message", "bot_response")

# TODO: Add your own examples here!
# Ideas:
# - Multi-step reasoning agent
# - Tool-using agent (calculator, search, etc.)
# - Agent with memory and learning
# - Multi-agent collaboration